# _In silico_ Reaction

## Table of Content <a class="anchor" id="toc"></a>
#### [Summary](#overview)
#### Data
* [Globals](#globals)
* [Compounds](#compounds)
* [Rules](#rules)
* [Reactions](#reactions)

#### Processing
* [Standardization](#standardization)
* [Rule Application](#prediction)
* [Workflow](#workflow)

#### Post-Processing
* [Exclusion Filters](#filter)

## <a class="anchor" id="overview"></a>Summary [$\Uparrow$](#toc)
This is the first major step in the workflow creating enviLink. Here, all substrates of the respective dataset's reactions are combined with all Eawag-BBD biotransformation rules (btrules), and subjected to the _in silico_ reactor [Ambit2](http://ambit.sourceforge.net/reactor.html). The predicted reactions are then collected and stored in a relational database for further processing, i.e. application of the [matching algorithm](match-KEGG.ipynb).

Before the reaction prediction, the substrates are submitted to various standardization routines, which identify standardized forms of the substrate (if they differ from the original form). And after the prediction the predicted reaction's products are submitted to the standardization routines as well. For more information and details about the standardization routines, see [Standardization Methods](./standardization%20methods.ipynb). All standardized forms are then stored in the mentioned database as well. In the second step of the enviLink workflow, [match KEGG](./match-KEGG.ipynb), the standardized forms together with the original molecule are combined to an equivalence class of molecules. This equivalence classes, rather then the individual molecules, will then be the essential input for the matching algorithm.

After the prediction, some of the predicted reactions are excluded from the list, because their products or substrates offend an additional, SMART-based restriction that is associated with some btrules.

## <a class="anchor" id="globals"></a>Globals [$\Uparrow$](#toc)
#### The Configuration
Containing credentials to access envipath and the workflow database.

In [1]:
import yaml
with open("config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

#### Directories Used in the Workflow

In [2]:
DATASET = 'EAWAG-BBD'
DATA = config['datadir'][DATASET]
TEMP = config['datadir']['temp']
BIN = config['binaries']
SRC = config['sources']

#### envipath Client
Used to collect the biodegradation rules

In [3]:
from envirest import EnviPathClient
from getpass import getpass
envipath = config['envipath']
client = EnviPathClient(
    envipath['host'],
    secure=envipath['secure'],
    verify=envipath['verified'])

#### Workflow Database
Stores normalized SMILES, rules, reactions to be matched and the  reactions from the _in silico_ reaction prediction. The database is initiated as an empty database and all tables are created along the workflow.<br>
The content of the database is the main output from the first step of the workflow and the main input for the second step.

In [4]:
from sqlalchemy import create_engine
database = config['db']
DRIVER = database['driver']
HOST = database['host']
USER = database['user']
DB = database['name'][DATASET]
PW = database['password']
rradb = create_engine(f'{DRIVER}://{USER}:{PW}@{HOST}/{DB}')

In [5]:
import sys
sys.path.append(f'{SRC}/rr')
from reacdb.sql import DataBase
params = {'host': HOST,
          'dbname': DB,
          'username': USER,
          'password': PW
         }
db = DataBase(params)

## <a class="anchor" id="compounds"></a>Compounds  [$\Uparrow$](#toc)
Compound input data is read from a tab separated text file (compounds.tsv) which has 3 columns:
- compound identifier
- compound group identifier
- the original (non standardized) SMILES string

The `compound group identifier` can be used when a database compound has alternative structures or in cases where the structure of a compound has some degree of freedom, like PCBs (polychlorinated biphenyls), where the exact position of the chloride atoms differs between stereoisomers.
<img src='Polychlorinated_biphenyl_structure.svg.png'>
The prediction routine can only work on single SMILES which are associated to the first column, `compound identifier`, but the matching algorithm eventually matches equivalence classes of compounds, which are based on the `compound group identifier`.

#### Read Compounds from File

In [6]:
from pandas import read_csv
RAWCOMPOUNDS = f'{DATA}/{DATASET}_compounds.tsv'
structures = read_csv(RAWCOMPOUNDS, sep='\t', header=None, names=['sid', 'cid', 'smiles'])
structures.head()

,sid,cid,smiles
0,8d63dbac-227d-4898-86fe-b19bf6683e84,3a797060-38fa-4661-8603-48773b3aff58,C1=CC2=C(C=C1)C3=C(C=C[C@@H]([C@@H]3O)O)C2
1,8723c215-b1e7-4719-9aa0-d4c1c85a0269,60de31b0-a3c3-4739-9f88-7c5c78a14a76,C=C(C)[C@H]1CC=C(C)CC1
2,5f574ab2-4990-4548-81e7-16bb217447ac,e4fe0464-864c-4cb3-9587-5a82d6dc67fa,CC1(C)C2CCC1(C)C(=O)C2
3,794de362-0e07-4a9d-a447-55dc59318b1f,04466823-ecf7-4dbd-9a81-ea0bf8e40dcc,CC(C)[C@@H]1CC[C@@H](C)CC1O
4,7640df0d-0730-4354-ad81-dfc6d668a81e,ff1575d1-ebba-4d8b-b5db-f4bae1049779,CC(C)[C@@H]1CC[C@@H](C)CC1=O


#### Normalization
Normalization removes co-enzyme A from the compounds because CoA groups are producing a high number of predicted reactions that do not contribute to a meaningful association between reactions. Technically, the normalization is a particular standardization routine (for details see [Standardization Methods](./standardization%20methods.ipynb)).

In [7]:
import subprocess
NORMCOMPOUNDS = f'{TEMP}/normcompounds.tsv'
STANDARDIZE = f'{BIN}/standardize'

command = f'{STANDARDIZE} -s cutCoA -f 3 -o {NORMCOMPOUNDS} {RAWCOMPOUNDS}'
ran = subprocess.run(command.split(), capture_output=True)
print(ran.stdout.decode())

0    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading global configuration
4    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading defaults: jar:file:/home/schema/work/bin/lib/PPS-tool-box-0.3.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.default-properties
4    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading artefact configuration: jniinchi-1.03_1
5    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading instance defaults: jar:file:/home/schema/work/bin/lib/PPS-tool-box-0.3.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.instance.default-properties
8    [main] INFO  net.sf.jnati.deploy.repository.ClasspathRepository  - Searching classpath for: jniinchi-1.03_1-LINUX-AMD64
9    [main] INFO  net.sf.jnati.deploy.repository.LocalRepository  - Searching local repository for: jniinchi-1.03_1-LINUX-AMD64
9    [main] DEBUG net.sf.jnati.deploy.repository.LocalRepository  - Artefact path: /home/schema/.jnati/repo/jniinchi/1

In [8]:
normcompounds = read_csv(NORMCOMPOUNDS, sep="\t", header=None, names=['sid', 'cid', 'smiles'])
normcompounds.head()

,sid,cid,smiles
0,8d63dbac-227d-4898-86fe-b19bf6683e84,3a797060-38fa-4661-8603-48773b3aff58,C1=CC2=C(C=C1)C3=C(C=C[C@@H]([C@@H]3O)O)C2
1,8723c215-b1e7-4719-9aa0-d4c1c85a0269,60de31b0-a3c3-4739-9f88-7c5c78a14a76,C=C(C)[C@H]1CC=C(C)CC1
2,5f574ab2-4990-4548-81e7-16bb217447ac,e4fe0464-864c-4cb3-9587-5a82d6dc67fa,CC1(C)C2CCC1(C)C(=O)C2
3,794de362-0e07-4a9d-a447-55dc59318b1f,04466823-ecf7-4dbd-9a81-ea0bf8e40dcc,CC(C)[C@@H]1CC[C@@H](C)CC1O
4,7640df0d-0730-4354-ad81-dfc6d668a81e,ff1575d1-ebba-4d8b-b5db-f4bae1049779,CC(C)[C@@H]1CC[C@@H](C)CC1=O


#### Upload normalized SMILES to the database

In [9]:
from sqlalchemy.exc import IntegrityError
for smiles in normcompounds.smiles.values:
    try:
        rradb.execute(f"insert into nsmiles (smiles) values ('{smiles}')")
    except IntegrityError:
        pass

#### Remark
Because of the removal of co-enzyme A, it is possible that multiple compounds have a common normalized SMILES.

In [10]:
normcompounds.groupby('smiles').count().sort_values('cid').tail()

,sid,cid
smiles,,
C1=C(C=CC(=C1)F)C(=O)[O-],3,3
C1=COC(=C1)C(=O)[O-],3,3
CC(=CCCC(C)CC(=O)[O-])C,3,3
C1=CC=C(C=C1)C(=O)CC(=O)[O-],3,3
C1=CSC(=C1)C(=O)[O-],3,3


#### Add Database IDs

In [11]:
from pandas import read_sql
nsmiles = read_sql('select * from nsmiles', rradb)
normcompounds = normcompounds.merge(nsmiles, on='smiles')
normcompounds.head()

,sid,cid,smiles,id
0,8d63dbac-227d-4898-86fe-b19bf6683e84,3a797060-38fa-4661-8603-48773b3aff58,C1=CC2=C(C=C1)C3=C(C=C[C@@H]([C@@H]3O)O)C2,1
1,8723c215-b1e7-4719-9aa0-d4c1c85a0269,60de31b0-a3c3-4739-9f88-7c5c78a14a76,C=C(C)[C@H]1CC=C(C)CC1,2
2,5f574ab2-4990-4548-81e7-16bb217447ac,e4fe0464-864c-4cb3-9587-5a82d6dc67fa,CC1(C)C2CCC1(C)C(=O)C2,3
3,794de362-0e07-4a9d-a447-55dc59318b1f,04466823-ecf7-4dbd-9a81-ea0bf8e40dcc,CC(C)[C@@H]1CC[C@@H](C)CC1O,4
4,7640df0d-0730-4354-ad81-dfc6d668a81e,ff1575d1-ebba-4d8b-b5db-f4bae1049779,CC(C)[C@@H]1CC[C@@H](C)CC1=O,5


In [12]:
structures.shape

(1507, 3)

#### Structure &rarr; Compound Standardization
A mentioned above, a database compound may have alternative structures. To store the information that these structures belong to the same equivalence class the `compound group identifier` is used to create a link between alternative structures.<br>
Since the equivalence classes are built from the `standards` table in the second step of the workflow, the link between alternative structures is entered into that table as well, even though there is no standardization routine applied.<br>
The `standardizer` value for these links is `compound`.

In [13]:
group_representative = normcompounds.cid.drop_duplicates().index
structures = structures.merge(normcompounds, on=['sid', 'cid'], suffixes=['_raw', ''])\
.merge(normcompounds.loc[group_representative,['cid','smiles']], on='cid', suffixes=['', '_comp'])
structures.shape, structures.drop_duplicates().shape

((1507, 6), (1507, 6))

In [14]:
def compsmiles(row):
    if row.smiles != row.smiles_comp:
        try:
            rradb.execute(f'''insert into standards (compound, standardizer, standard)
            values (
                (select id from nsmiles where smiles = '{row.smiles}'),
                (select id from standardizer where name = 'compound'),
                (select id from nsmiles where smiles = '{row.smiles_comp}')
            )''')
            return 1
        except IntegrityError:
            return 0
    else:
        return None

try:
    rradb.execute("insert into standardizer (name) values ('compound')")
except IntegrityError:
    pass

inserts = structures[structures.smiles != structures.smiles_comp].apply(compsmiles, axis=1)
sum(inserts) if inserts.shape[0] else None

11

## <a class="anchor" id="rules"></a>Rules  [$\Uparrow$](#toc)

The btrules are part of the EAWAG-BBD package from envipath.org. They are downloaded with the envipath-api Client.

In [15]:
from pandas import DataFrame
BBD = client.findpackage('EAWAG-BBD')
bbdrules = DataFrame(client.get(f'{BBD}/rule')['rule'])
bbdrules.head()

,id,identifier,name,reviewStatus
0,https://envipath.org/package/32de3cf4-e3e6-416...,parallel-rule,bt0001,reviewed
1,https://envipath.org/package/32de3cf4-e3e6-416...,simple-rule,bt0001-3568,reviewed
2,https://envipath.org/package/32de3cf4-e3e6-416...,parallel-rule,bt0002,reviewed
3,https://envipath.org/package/32de3cf4-e3e6-416...,simple-rule,bt0002-3673,reviewed
4,https://envipath.org/package/32de3cf4-e3e6-416...,parallel-rule,bt0003,reviewed


In EAWAG-BBD all "simple" rules are grouped into "parallel" rules, which combine the individual SMIRKS to one _btrule_.<br>
Only the "simple" rules are active in the prediction and have a SMIRKS string encoding  reaction patterns.

In [16]:
from tqdm import tqdm
tqdm.pandas()


from pandas import Series
exceptions = []
def annotate_rule(row):
    try:
        rule = client.get(row.id)
        return Series({
            'smirks':rule['smirks'],
            'substrateFilter':rule.get('reactantFilterSmarts'),
            'productFilter':rule.get('productFilterSmarts'),
            'container':[cr['id'] for cr in rule.get('includedInCompositeRule') if cr['id'][:66] == row.id[:][:66]][0]
        })
    except Exception as e:
        exceptions.append([row.id, e.__class__.__name__, str(e)])

ANNOT_RULES = f'{TEMP}/envipath-rules.annotated.tsv'
try:
    rules
except NameError:
    try:
        rules = read_csv(ANNOT_RULES, sep='\t')
    except FileNotFoundError:
        rules = bbdrules[bbdrules.identifier == 'simple-rule'].loc[:,['id', 'name']]
        rules = rules.join(rules.progress_apply(annotate_rule, axis=1))
rules.tail()

100%|██████████| 291/291 [00:16<00:00, 17.57it/s]


,id,name,smirks,substrateFilter,productFilter,container
490,https://envipath.org/package/32de3cf4-e3e6-416...,bt0438-4230,[#8-:11]-[#6:9](=[O:10])-[#6:1]([H])([H])-[c:2...,,,https://envipath.org/package/32de3cf4-e3e6-416...
492,https://envipath.org/package/32de3cf4-e3e6-416...,bt0439-4270,"[H:14][C:3]([#1,#6;A:13])([#6:10](-[#8-:11])=[...",,,https://envipath.org/package/32de3cf4-e3e6-416...
494,https://envipath.org/package/32de3cf4-e3e6-416...,bt0440-4255,"[#7:1][P:2]([#6,#7,#8,#16:4])([#6,#8:5])=[#8,#...",,,https://envipath.org/package/32de3cf4-e3e6-416...
496,https://envipath.org/package/32de3cf4-e3e6-416...,bt0443-4291,[#6:5]-[#6:1]([H])(-[#8:2]([H]))C#N>>[#6:5]-[#...,,,https://envipath.org/package/32de3cf4-e3e6-416...
498,https://envipath.org/package/32de3cf4-e3e6-416...,bt0444-4310,[H][#6:3](-[#6:2](-[#7:1]([H])-[#6:7](-[#6:9]-...,,,https://envipath.org/package/32de3cf4-e3e6-416...


#### Unique rule names
The workflow assumes unique rule names. Otherwise there will be inconsistencies.

In [17]:
assert rules.name.nunique() == rules.shape[0]

#### Write rules to file
Two files are created, envipath-rules.tsv, which is used in the prediction step, and envipath-rules.annotated.tsv containing additional information for later reference.<br>
envipath-rules.tsv contains only "simple" rules.

In [18]:
rules.to_csv(ANNOT_RULES, sep="\t", index=None)
RULES = f'{TEMP}/envipath-rules.tsv'
rules.loc[:,['name','smirks']].to_csv(RULES, sep="\t", index=None, header=None)
read_csv(RULES, sep="\t", header=None).head()

,0,1
0,bt0001-3568,"[H][#8:2][C:1]([H:5])([H])[#1,#6:6]>>[H:5][#6:..."
1,bt0002-3673,[H][#8:1][#6;A;!$(CCC(O)[O-]):2]([H])([#6:5])[...
2,bt0003-1196,[H][#6:1](-[#6:5])=[O:4]>>[#6:5]-[#6:1](-[#8-]...
3,bt0005-3667,[#8:7]([H])-[#6:1]([H])-1-[#6:2]=[#6:3]-[#6:4]...
4,bt0005-3776,[c:7]([H])1[c:8]([H])[c:9]([H])[c:10]2[c:11]([...


#### Insert Rules into Database
The database is used for tracking the predicted reactions and not for the prediction itself. Hence the rules table of the database is not required to contain the SMIRKS strings of the rules.<br>
But opposite to envipath-rules.tsv, it contains the information about the _btrule_ to which the simple rule belongs.

In [19]:
def splitrule(r):
    a = r.split('-')
    return Series({'container':'-'.join(a[:-1]),'simple':a[-1]})
rulecontainers = rules.apply(lambda row: splitrule(row['name']), axis=1)
rulecontainers.to_csv(f'{TEMP}/importrules.csv', header=None, index=None)
rulecontainers.loc[:,['container','container']].drop_duplicates().to_csv(f'{TEMP}/importrulecontainers.csv', header=None, index=None)

In [20]:
rule_import_df = read_csv(f'{TEMP}/importrulecontainers.csv', header=None)\
         .append(read_csv(f'{TEMP}/importrules.csv', header=None, dtype=str))
rule_import_df.columns = ['container', 'simple']
rule_import_df = rule_import_df.merge(
    read_sql('select * from rule', rradb),
    on=['container', 'simple'], how='left')
rule_import_df[rule_import_df.id.isnull()]\
    .loc[:,['container', 'simple']]\
    .to_sql('rule', rradb, index=None, if_exists='append', method='multi')

## <a class="anchor" id="reactions"></a>Reactions  [$\Uparrow$](#toc)

Reaction input data is read from a tab separated text file (reactions.tsv) which has 3 columns:
- substrate SMILES
- reaction identifier
- product SMILES

Both substrates and products can contain multiple molecules, in which case the individual SMILES are separated by a '.' in accordance with Daylight's SMILES syntax.

#### Normalization
Analogously to the compound import, CoA is removed from all substrates and products.

In [21]:
RAWREACTIONS = f'{DATA}/{DATASET}_reactions.tsv'
NORMREACTIONS = f'{DATA}/{DATASET}_normreactions.tsv'
STANDARDIZE = f'{BIN}/standardize'

command = f'{STANDARDIZE} -s cutCoA -f 1,3 -o {NORMREACTIONS} {RAWREACTIONS}'
ran = subprocess.run(command.split(), capture_output=True)
print(ran.stdout.decode())

0    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading global configuration
4    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading defaults: jar:file:/home/schema/work/bin/lib/PPS-tool-box-0.3.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.default-properties
5    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading artefact configuration: jniinchi-1.03_1
6    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading instance defaults: jar:file:/home/schema/work/bin/lib/PPS-tool-box-0.3.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.instance.default-properties
8    [main] INFO  net.sf.jnati.deploy.repository.ClasspathRepository  - Searching classpath for: jniinchi-1.03_1-LINUX-AMD64
10   [main] INFO  net.sf.jnati.deploy.repository.LocalRepository  - Searching local repository for: jniinchi-1.03_1-LINUX-AMD64
10   [main] DEBUG net.sf.jnati.deploy.repository.LocalRepository  - Artefact path: /home/schema/.jnati/repo/jniinchi/1

In [22]:
reactions = read_csv(NORMREACTIONS, sep='\t', header=None, names=['substrates', 'reaction', 'products'])
print(reactions.shape)
reactions.head()

(1479, 3)


,substrates,reaction,products
0,C1=C(C=CC(=C1)O)C(C2=CC=C(C=C2)O)O,00549813-a13d-442f-a963-1b146cfb2df5,C1=C(C=CC(=C1)O)C(=O)C2=CC=C(C=C2)O
1,C1=CC=C(C=C1)OC2=CC(=CC=C2)C(C#N)O,0066b7c0-544b-47fd-9b93-8a8b275adead,C1=CC=C(C=C1)OC2=CC(=CC=C2)C=O
2,CCCC[Sn+2]CC(CC)O,00a0fe94-d694-4913-866f-1c9253fcc67a,CCC(=O)C.CCCC[Sn+3]
3,CC1=CC=C(C=C1)C(=O)[O-],00b9590f-c0c9-4506-ab7e-52d73364d996,CC1=C[C@@H]([C@](C=C1)(C(=O)[O-])O)O
4,C1=C[C@H]([C@H](C(=C1)CCC(=O)[O-])O)O,0104a8a5-b682-4664-9495-52feb2af184a,C1=CC(=C(C(=C1)CCC(=O)[O-])O)O


#### Insert Reactions into Database

In [23]:
def _split_reaction(reacdf):
    roles = []
    reaction_labels = []
    def splitrow(row):
        from re import split
        '''
        expects a line in this format:
        <SMILES[.SMILES]>\\t<label|\\[label[, \[label\]]>\\t<SMILES[.SMILES]>
        '''
        s,l,p = row[0], row[1], row[2]
        
        substrates = s.split(".")
        products = p.split(".")
        for s in substrates:
            roles.append([row.name, s, False])
        for p in products:
            roles.append([row.name, p, True])
            
        labels = split(r'\s*,\s*',l.strip(r"[\[\]]")) if l[0]=="[" and l[-1]=="]" else [l]
        for label in labels:
            reaction_labels.append([row.name, label])
    
    reacdf.apply(splitrow, axis=1)
    
    # not drop duplicates on labels but not on roles: A>>B.B != A>>B, but A>L,L>B = A>L>B
    return DataFrame(roles, columns=['reacid', 'smiles', 'isproduct']),\
           DataFrame(reaction_labels, columns=['reacid', 'label']).drop_duplicates()


def _get_compound_ids(rradb, role_df):
    nsmiles = read_sql('''
    select id compound, smiles 
    from nsmiles
    ''', rradb)
    role_id_df = role_df.merge(nsmiles, on='smiles', how='left')

    # insert novel smiles
    role_id_df[role_id_df['compound'].isnull()]\
        .loc[:,['smiles']]\
        .drop_duplicates()\
        .to_sql('nsmiles', rradb, index=None, if_exists='append', method='multi')

    nsmiles = read_sql('''
    select id compound, smiles 
    from nsmiles
    ''', rradb)
    role_id_df = role_df.merge(nsmiles, on='smiles', how='left')    
    role_id_df['compound'] = role_id_df['compound'].astype(str)
    
    return role_id_df


def _get_reaction_ids(rradb, role_id_df):
    educts = role_id_df[~role_id_df.isproduct].groupby('reacid').agg({'compound': ' '.join}).rename(columns={'compound':'substrates'})
    products = role_id_df[role_id_df.isproduct].groupby('reacid').agg({'compound': ' '.join}).rename(columns={'compound':'products'})
    reaction_df = educts.join(products)
    reaction_df['reacid'] = reaction_df.index
    reaction_df['signature'] = reaction_df.apply(
        lambda row:
            db.signature([int(_) for _ in row.substrates.split()], 
                         [int(_) for _ in row.products.split()]),
        axis=1)

    reactions = read_sql('''
    select id reaction, signature
    from reaction
    ''', rradb)
    reaction_id_df = reaction_df.merge(reactions, on='signature', how='left')

    # insert novel reactions
    novel_reactions = reaction_id_df[reaction_id_df.reaction.isnull()]\
        .loc[:,['signature']]\
        .drop_duplicates()\
        .copy()
    novel_reactions.to_sql('reaction', rradb, index=None, if_exists='append', method='multi')

    reactions = read_sql('''
    select id reaction, signature
    from reaction
    ''', rradb)
    reaction_id_df = reaction_df.merge(reactions, on='signature', how='left')
    
    role_id_df.merge(
        reaction_id_df[reaction_id_df.signature.isin(novel_reactions.signature)],
        on='reacid'
    ).loc[:,['compound', 'reaction', 'isproduct']]\
        .to_sql('rrole', rradb, index=None, if_exists='append', method='multi')
    
    return reaction_id_df

In [24]:
def insert_edb_reactions(rradb, reacdf):
    role_df, reaction_label_df = _split_reaction(reacdf)

    #### reaction role dataframe
    role_id_df = _get_compound_ids(rradb, role_df)

    #### reaction dataframe
    reaction_id_df = _get_reaction_ids(rradb, role_id_df)

    # adjust columns
    reaction_label_df = reaction_label_df.rename(columns={'label':'envipath_url'})
    reaction_label_df['id'] = reaction_label_df.apply(lambda row: f'g{row.reacid:04d}', axis=1)

    # insert rule reaction links
    edb_reaction_id_df = reaction_label_df\
        .merge(reaction_id_df, on='reacid')\
        .loc[:,['id', 'reaction', 'envipath_url']]\
        .drop_duplicates()
    
    edb_reactions = read_sql('''
    select id, reaction, envipath_url, 1 e
    from edb_reaction
    ''', rradb)
    
    edb_reaction_id_df = edb_reaction_id_df\
        .merge(edb_reactions, on=['id', 'reaction', 'envipath_url'], how='left')
    edb_reaction_id_df[edb_reaction_id_df.e.isnull()]\
        .loc[:,['id', 'reaction', 'envipath_url']]\
        .to_sql('edb_reaction', rradb, index=None, if_exists='append', method='multi')

In [25]:
insert_edb_reactions(rradb, read_csv(NORMREACTIONS, header=None, sep="\t"))
read_sql('select count(*) from edb_reaction', rradb)

,count
0,1479


## <a class="anchor" id="standardization"></a>Standardization  [$\Uparrow$](#toc)
Six different standardization methods are applied, first to all substrates and products of the dataset's reactions and then, after the prediction, to all predicted products.<br>
A detailed description of these methods can be found [here](standardization%20methods.ipynb).
#### Register the Standardizers in the Database

In [26]:
STANDARDIZERS = ["basic","flatten","enhanced","cistrans","exotic","enolketo"]
for standardizer in STANDARDIZERS:
    try:
        rradb.execute(f"insert into standardizer (name) values ('{standardizer}')")
    except IntegrityError:
        pass

#### Define Standardization Routines

In [27]:
def multiprocessing_standardization(smiles_collection, tempdir=TEMP, runb=STANDARDIZE):
    if len(list(smiles_collection)) == 0:
        return DataFrame(columns=['norm', 'standardized', 'standardizer'])
    
    std_df = DataFrame({'norm': list(smiles_collection)})
    std_df.to_csv(f'{tempdir}/norm.compounds', sep="\t", header=None)
   
    standardized_list = []

    def assign(row, standardizer):
        standardized_list.append({
            'id': row.name, 'norm':row.norm, 'standardized': row[1], 'standardizer':standardizer
        })
    
    ps = dict()
    for standardizer in STANDARDIZERS:
        import os
        try:
            os.remove(f'{tempdir}/standardized.{standardizer}.compounds.tsv')
        except FileNotFoundError:
            pass
        
        command = f'''{runb}
            -f 2
            -s {standardizer}
            -o {tempdir}/standardized.{standardizer}.compounds.tsv
            {tempdir}/norm.compounds
        '''
        ps[standardizer] = {'process': subprocess.Popen(command.split())}
    
    while any([ps[standardizer].get('rc') == None for standardizer in STANDARDIZERS]):
        from time import sleep
        sleep(1)
        for standardizer in STANDARDIZERS:
            ps[standardizer]['rc'] = ps[standardizer]['process'].poll()
    
    for standardizer in STANDARDIZERS:
        stdout, stderr = ps[standardizer]['process'].communicate()
        ps[standardizer]['out'] = stdout
        ps[standardizer]['err'] = stderr
        if ps[standardizer]['rc'] != 0:
            raise Exception(ps[standardizer]['err'])
        
        stds = std_df.join(
            read_csv(f'{tempdir}/standardized.{standardizer}.compounds.tsv', sep="\t", header=None, index_col=0),
        )
        stds[stds[1].notnull() & (stds.norm!=stds[1])]\
            .apply(
                lambda row: assign(row, standardizer),
                axis=1
            )
    
    standardized_df = DataFrame(standardized_list).drop_duplicates()
    return standardized_df

In [28]:
def db_update_all_standards(rradb, stddf):
    if not stddf.shape[0]:
        return
    
    standard_smiles = stddf.standardized.drop_duplicates().values

    # collect known ids for standardized smiles
    present_nsmiles = read_sql(f'''
    select id stdid, smiles standardized
    from nsmiles
    where smiles in ('{"', '".join(standard_smiles)}')''', rradb)
    stddf_stdid = stddf.merge(present_nsmiles, on='standardized', how='left')

    # insert novel smiles
    stddf_stdid[stddf_stdid.stdid.isnull()]\
        .rename(columns={'standardized':'smiles','smiles':'nosmiles'})\
        .loc[:,'smiles']\
        .drop_duplicates()\
        .to_sql('nsmiles', rradb, index=None, if_exists='append', method='multi')


    # collect known ids for norm smiles
    present_nsmiles = read_sql(f'''
    select id normid, smiles norm
    from nsmiles
    where smiles in ('{"', '".join(stddf.norm.drop_duplicates().values)}')''', rradb)
    stddf_stdid = stddf.merge(present_nsmiles, on='norm', how='left')

    # once more collect ids for standardized smiles, now including novel smiles
    present_nsmiles = read_sql(f'''
    select id stdid, smiles standardized
    from nsmiles
    where smiles in ('{"', '".join(standard_smiles)}')''', rradb)
    stddf_stdid = stddf_stdid.merge(present_nsmiles, on='standardized', how='left')

    # collect standardizer ids
    standardizers = read_sql(f'''
    select id stdzrid, name standardizer
    from standardizer
    ''', rradb)
    stddf_stdid = stddf_stdid.merge(standardizers, on='standardizer')


    # collect known standardizations
    compounds = [str(i) for i in stddf_stdid.normid.drop_duplicates().values]
    known_standards = read_sql('''
    select compound normid, standard known_standard, standardizer stdzrid
    from standards
    where compound in ({})'''.format(",".join(compounds)), rradb)

    standards_df = stddf_stdid.merge(known_standards, on=['normid', 'stdzrid'], how='left')

    # consistency check
    if standards_df[standards_df.known_standard.notnull() & (standards_df.stdid != standards_df.known_standard)].shape[0]:
        print("something's wrong with these standardizations:")
        print(standards_df[standards_df.known_standard.notnull() & (standards_df.stdid != standards_df.known_standard)].shape)
        return standards_df

    # insert novel standardizations
    standards_df[standards_df.known_standard.isnull()]\
        .rename(columns={'normid': 'compound', 'stdid':'standard', 'stdzrid':'standardizer', 'standardizer':'name'})\
        .loc[:,['compound', 'standard', 'standardizer']]\
        .to_sql('standards', rradb, index=None, if_exists='append', method='multi')


In [29]:
def db_update_all_status(rradb, smileslist, propagated=False, standardized=False):
        # once more collect ids for standardized smiles, now including novel smiles
    present_nsmiles = read_sql(f'''
    select id, smiles, standardized, propagated
    from nsmiles natural left join progress
    where smiles in ('{"', '".join(smileslist)}')''', rradb)
    
    # update existing entries
    oldprogress = present_nsmiles[present_nsmiles.standardized.notnull()]
    if oldprogress.shape[0] > 0:
        idlist = ",".join([str(_) for _ in oldprogress.id.values])
        if propagated:
            rradb.execute(f'update progress set propagated = TRUE where id in ({idlist})')
        if standardized:
            rradb.execute(f'update progress set standardized = TRUE where id in ({idlist})')
    
    # insert new entries
    newprogress = present_nsmiles[present_nsmiles.standardized.isnull()].index
    present_nsmiles.loc[newprogress,'standardized'] = standardized
    present_nsmiles.loc[newprogress,'propagated'] = propagated
    present_nsmiles.loc[newprogress,['id', 'standardized', 'propagated']]\
        .to_sql('progress', rradb, if_exists='append', method='multi', index=None)

In [30]:
def standardize_unstandardized(rradb):
    nonstandardized = read_sql('''
    select id, smiles
    from nsmiles natural left join progress
    where not standardized or standardized is NULL
    ''', rradb).smiles.drop_duplicates().values
    print('nonstandardized', len(nonstandardized))
    
    standardized = multiprocessing_standardization(nonstandardized)
    print('standardized', standardized.shape)
    
    db_update_all_standards(rradb, standardized)
    print('standards updated')
    
    db_update_all_status(rradb, nonstandardized, standardized=True)
    print('standardization status updated')

## <a class="anchor" id="prediction"></a>Rule Application  [$\Uparrow$](#toc)

Rules are applied to all substrates of the original reactions and to their standardized forms. This is done through the script _runAndCompare_, which in turn calls the _main_ method of the class _org.kramerlab.pps.api.tools.App_. The cell below shows the command line help text for this script:

In [31]:
PROPAGATION = f'{BIN}/runAndCompare'
command = [PROPAGATION, '--help']
helptext = subprocess.run(command, capture_output=True)
print(helptext.stdout.decode().strip())

Usage: ../bin/runAndCompare -x rules -s substrates [-r reactions] [-o outdir]
where
- rules is a tab delimited file with 2 columns: rule-id and SMARTS strings
- substrates is a file with SMILES strings
- reactions is a tab delimited file with 3 columns: substrate SMILES, rule-id(s), product SMILES (joined by '.' if there are more than one)

runs all rules on all substrates, the results are then compared to the given reactions
in the outdir directory three files are created: 
- unmatched_reactions.csv, not resulting from rules and substrates but present in the reactions file
- matched_reactions.csv, resulting from rules and substrates and present in the reactions file
- new_reactions.csv, resulting from rules and substrates but not present in the reactions file


#### Insert the Ambit2 Reactor into the Database
The data model of _enviLink_ requires a name for the reactor. In another context this could be used to systematically compare products of different reactors. In the present context there is only one reactor, Ambit2. Nevertheless it must be registered in the database.

In [32]:
AMBIT = 1
try:
    rradb.execute(f"insert into reactor (id, name) values ({AMBIT}, 'ambit')")
except IntegrityError:
    pass

#### Define Rule Application Routines

In [33]:
PROPAGATION = f'{BIN}/runAndCompare'
from shutil import rmtree
from os import makedirs
        
def multiprocessing_propagation(smiles_collection, rules_df, nworker, tempdir=TEMP, runb=PROPAGATION):
    if nworker < 1:
        raise Exception('need at least one worker!')
    
    if len(list(smiles_collection)) == 0:
        return DataFrame(columns=['norm', 'standardized', 'standardizer'])

    smiles_df = DataFrame({'norm': list(smiles_collection)})
    rules_df.loc[:,['name','smirks']].to_csv(f'{tempdir}/rules.tsv', sep="\t", header=None, index=None)
    
    workers = []
    for i in range(nworker):
        # clean out
        try:
            rmtree(f'{tempdir}/p{i}')
        except FileNotFoundError:
            pass
        makedirs(f'{tempdir}/p{i}')
        
        # write compound fils
        smiles_df[i::nworker].to_csv(f'{tempdir}/p{i}/compounds.tsv', sep="\t", header=None, index=None)
        
        # run prediction (call java programm)
        command = f'''{runb}
            -x {tempdir}/rules.tsv
            -s {tempdir}/p{i}/compounds.tsv
            -o {tempdir}/p{i}
        '''

        workers.append({'process': subprocess.Popen(command.split())})
    
    while any([workers[i].get('rc') == None for i in range(nworker)]):
        from time import sleep
        sleep(1)
        for i in range(nworker):
            workers[i]['rc'] = workers[i]['process'].poll()
    
    reactions_df = DataFrame()
    for i in range(nworker):
        stdout, stderr = workers[i]['process'].communicate()
        workers[i]['out'] = stdout
        workers[i]['err'] = stderr
        if workers[i]['rc'] != 0:
            raise Exception(workers[i]['err'])
        
        reactions_df = reactions_df.append(
            read_csv(f'{tempdir}/p{i}/new_reactions.csv', sep="\t", header=None)
        )
    
    # it's possible that the smiles from the resulting reaction differs from the original substrate
    # e.g. CNC(=O)OC1=C(/C=C\C(=O)[O-])C(=CC=C1)/C(=O)[O-]
    #      CNC(=O)OC1=C(/C=C\C(=O)[O-])C(=CC=C1)C(=O)[O-]
    # (check $ diff <( cut -f2 compounds.tsv | sort ) <( cut -f 1 new_reactions.csv | sort | uniq ))
    return reactions_df.drop_duplicates().reset_index(drop=True)

In [34]:
def db_update_all_propagation(rradb, db, reacdf, reactorname='ambit'):
    role_df,  rule_reaction_df = _split_reaction(reacdf)

    #### reaction role dataframe
    role_id_df = _get_compound_ids(rradb, role_df)
    
    #### reaction dataframe
    reaction_id_df = _get_reaction_ids(rradb, role_id_df)
    
    #### rule reaction dataframe
    rule_reaction_df['reactor'] = AMBIT

    # adjust column names
    rule_reaction_df = rule_reaction_df.rename(columns={'label':'btrule'})
    
    # collect rules
    rules = read_sql('''
    select id, container||'-'||simple btrule
    from rule
    ''', rradb)
    rules = rules.rename(columns={'id':'rule'})

    # insert rule reaction links
    rule_reaction_id_df = rule_reaction_df\
        .merge(rules, on='btrule')\
        .merge(reaction_id_df, on='reacid')\
        .loc[:,['reaction', 'rule', 'reactor']]\
        .drop_duplicates()
    
    rule_reactions = read_sql('''
    select reaction, rule, reactor, 1 e
    from rule_reaction
    ''', rradb)
    
    rule_reaction_id_df = rule_reaction_id_df\
        .merge(rule_reactions, on=['reaction', 'rule', 'reactor'], how='left')
    rule_reaction_id_df[rule_reaction_id_df.e.isnull()]\
        .loc[:,['reaction', 'rule', 'reactor']]\
        .to_sql('rule_reaction', rradb, index=None, if_exists='append', method='multi')

In [35]:
def predict_unpredicted(rradb, db, nworkers=4, limit=None):
    querylimit = f'limit {limit}' if limit else ''
    nonpropagated = read_sql(f'''
    select id, smiles
    from nsmiles natural left join progress
    where not propagated or propagated is NULL
    {querylimit}''', rradb)
    print('nonpropagated', nonpropagated.shape)
    
    propagated = multiprocessing_propagation(nonpropagated.smiles.drop_duplicates().values, rules, nworkers)
    print('propagated', propagated.shape)
    
    db_update_all_propagation(rradb, db, propagated)
    print('propagated updated')
    
    db_update_all_status(rradb, nonpropagated.smiles.values, propagated=True)
    print('propagation status updated')

## <a class="anchor" id="workflow"></a>Workflow  [$\Uparrow$](#toc) <a style="color:red"> Ich verstehe nicht, was der Unterschied zwiscshen Rule Application und Workflow ist. Wo werden die Regeln denn nun angewendet? Oben oder hier?</a> 
The _in silico_ reaction workflow consists of three steps. At first all compounds undergo an initial standardization, then the rules are applied and at last the yet unstandardized products are submitted to another standardization step.

## <a class="anchor" id="workflow"></a>Workflow  [$\Uparrow$](#toc) </a>
So far, standardization and prediction routines have only been defined. Now they are executed in consecutive order.<br>
First, all compounds are standardized, then the rules are applied and at last the yet unstandardized products are submitted to another standardization.

In [36]:
standardize_unstandardized(rradb)
predict_unpredicted(rradb, db)
standardize_unstandardized(rradb)

nonstandardized 1367
standardized (413, 4)
standards updated
standardization status updated
nonpropagated (1701, 2)
propagated (9937, 3)
propagated updated
propagation status updated
nonstandardized 7613
standardized (3158, 4)
standards updated
standardization status updated


## <a class="anchor" id="filter"></a>Exclusion Filters  [$\Uparrow$](#toc)
In enviPath, rules can have exclusion filters that inactivate them based on SMARTS patterns of the substrates or products. The prediction routine _runAndCompare_, which was applied here, does not take these filters into account. Therefore, reactions breaking the exclusion constraints are identified separately here and marked as excluded in the database.

#### Read Exclusion Filters from File

In [37]:
filtruls = read_csv(f'{TEMP}/envipath-rules.annotated.tsv', sep='\t')
filtruls.head()

,id,name,smirks,substrateFilter,productFilter,container
0,https://envipath.org/package/32de3cf4-e3e6-416...,bt0001-3568,"[H][#8:2][C:1]([H:5])([H])[#1,#6:6]>>[H:5][#6:...",[$([H]C(C)=O)],NaN,https://envipath.org/package/32de3cf4-e3e6-416...
1,https://envipath.org/package/32de3cf4-e3e6-416...,bt0002-3673,[H][#8:1][#6;A;!$(CCC(O)[O-]):2]([H])([#6:5])[...,[$([O;H1]C1C=CC=CC1[O;H1])],NaN,https://envipath.org/package/32de3cf4-e3e6-416...
2,https://envipath.org/package/32de3cf4-e3e6-416...,bt0003-1196,[H][#6:1](-[#6:5])=[O:4]>>[#6:5]-[#6:1](-[#8-]...,NaN,NaN,https://envipath.org/package/32de3cf4-e3e6-416...
3,https://envipath.org/package/32de3cf4-e3e6-416...,bt0005-3667,[#8:7]([H])-[#6:1]([H])-1-[#6:2]=[#6:3]-[#6:4]...,NaN,NaN,https://envipath.org/package/32de3cf4-e3e6-416...
4,https://envipath.org/package/32de3cf4-e3e6-416...,bt0005-3776,[c:7]([H])1[c:8]([H])[c:9]([H])[c:10]2[c:11]([...,NaN,NaN,https://envipath.org/package/32de3cf4-e3e6-416...


In [38]:
def splitname(row):
    container, simple = row['name'].split('-')
    return Series({'btrule':container, 'simple':simple})
filtruls = filtruls.join(filtruls.apply(splitname, axis=1))
filtruls.head()

,id,name,smirks,substrateFilter,productFilter,container,btrule,simple
0,https://envipath.org/package/32de3cf4-e3e6-416...,bt0001-3568,"[H][#8:2][C:1]([H:5])([H])[#1,#6:6]>>[H:5][#6:...",[$([H]C(C)=O)],NaN,https://envipath.org/package/32de3cf4-e3e6-416...,bt0001,3568
1,https://envipath.org/package/32de3cf4-e3e6-416...,bt0002-3673,[H][#8:1][#6;A;!$(CCC(O)[O-]):2]([H])([#6:5])[...,[$([O;H1]C1C=CC=CC1[O;H1])],NaN,https://envipath.org/package/32de3cf4-e3e6-416...,bt0002,3673
2,https://envipath.org/package/32de3cf4-e3e6-416...,bt0003-1196,[H][#6:1](-[#6:5])=[O:4]>>[#6:5]-[#6:1](-[#8-]...,NaN,NaN,https://envipath.org/package/32de3cf4-e3e6-416...,bt0003,1196
3,https://envipath.org/package/32de3cf4-e3e6-416...,bt0005-3667,[#8:7]([H])-[#6:1]([H])-1-[#6:2]=[#6:3]-[#6:4]...,NaN,NaN,https://envipath.org/package/32de3cf4-e3e6-416...,bt0005,3667
4,https://envipath.org/package/32de3cf4-e3e6-416...,bt0005-3776,[c:7]([H])1[c:8]([H])[c:9]([H])[c:10]2[c:11]([...,NaN,NaN,https://envipath.org/package/32de3cf4-e3e6-416...,bt0005,3776


#### Substrate Filters

In [39]:
REACTANT_FILTERS = f'{TEMP}/reactantFilters.tsv'

reactionFilters = filtruls[filtruls.substrateFilter.notnull()]\
    .loc[:,['simple', 'substrateFilter']]
reactionFilters\
    .to_csv(REACTANT_FILTERS, header=None, index=None, sep='\t')
reactionFilters.head()

,simple,substrateFilter
0,3568,[$([H]C(C)=O)]
1,3673,[$([O;H1]C1C=CC=CC1[O;H1])]
5,4282,"[$([#1,#8]~C(C([#8-])=O)c1ccccc1C([#8-])=O),$(..."
6,4163,"[$([O-][N+](=O)c1ccccc1),$([#6]-c1[c;H1][c;H1]..."
8,4164,"[$(Cc1cc(*):nc(*):c1),$([O-][N+](=O)c1cccc(c1)..."


In [40]:
REACTANT_FILTER_SUBSTRATES = f'{TEMP}/reactantFilterSubstrates.tsv'

ruleList = "','".join(reactionFilters.simple.values)
affectedByReactionFilters = read_sql(f'''
select distinct reaction, simple, smiles
from rule_reaction
 join rule on rule.id = rule
 join rrole using(reaction)
 join nsmiles on nsmiles.id = compound
where isproduct is FALSE 
 and reactor = 1
 and simple in ('{ruleList}')
''', rradb)
affectedByReactionFilters\
    .to_csv(REACTANT_FILTER_SUBSTRATES, header=None, index=None, sep='\t')
affectedByReactionFilters.head()

,reaction,simple,smiles
0,6289,2982,C(=C\C(=O)C(=O)[O-])/C1=CC(=CC=C1O)N
1,5872,3938,CCC1=CC=CC(=C1NCOC)CC
2,6021,3938,CN1[C@@H]2C[C@H]1C([C@@H]([C@@H](C2)O)C(=O)[O-])O
3,9467,4165,C1=CC=C(C=C1)C#N
4,8706,3854,C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O[C@@H]2[...


In [41]:
reactionFilters.merge(affectedByReactionFilters, on='simple').groupby('simple').agg({'reaction':'count'})\
    .sort_values(by='reaction', ascending=False).head()

,reaction
simple,
4282,1495
3859,488
3673,421
4215,247
3938,221


In [42]:
POST_REACTION_FILTER = f'{BIN}/postReactionFilter'
REACTANT_FILTER_MATCHES = f'{TEMP}/reactantFilterMatches.tsv'

command = f'''{POST_REACTION_FILTER}
    -c {REACTANT_FILTER_SUBSTRATES}
    -p {REACTANT_FILTERS}
    -m {REACTANT_FILTER_MATCHES}'''
ran = subprocess.run(command.split(), capture_output=True)
print(ran.stdout.decode())

#### Product Filters

In [43]:
PRODUCT_FILTERS = f'{TEMP}/productFilters.tsv'

productFilters = filtruls[filtruls.productFilter.notnull()]\
    .loc[:,['simple', 'productFilter']]
productFilters\
    .to_csv(PRODUCT_FILTERS, header=None, index=None, sep='\t')
productFilters.head()

,simple,productFilter
5,4282,[$([#6]!@-c1cc([O;H1])c([O;H1])c2[c;H1][c;H1][...
8,4164,"[$(C!@-c1cccc(O)c1!@-C),$(Cc1ccnc(=O)c1),$(O=C..."
10,4165,[$(C!@-c1cc(O)c2ccccc2c1!@-C)]
12,4215,"[$(OC1=[C;R1]C2=C([C;R1]=C1O)[C,N]=CC=C2),$(Oc..."
54,4171,"[$(Oc1cc(:[c,n,o,s])c(:[c,n,o,s])cc1O)]"


In [44]:
PRODUCT_FILTER_SUBSTRATES = f'{TEMP}/productFilterSubstrates.tsv'

ruleList = "','".join(productFilters.simple.values)
affectedByProductFilters = read_sql(f'''
select distinct reaction, simple, smiles
from rule_reaction
 join rule on rule.id = rule
 join rrole using(reaction)
 join nsmiles on nsmiles.id = compound
where isproduct is TRUE 
 and reactor = 1
 and simple in ('{ruleList}')
''', rradb)
affectedByProductFilters\
    .to_csv(PRODUCT_FILTER_SUBSTRATES, header=None, index=None, sep='\t')
affectedByProductFilters.head()

,reaction,simple,smiles
0,21,4215,CC1=CC2=C(C(=CC=C2NC1=O)O)O
1,60,4215,C1=CC(=C(C2=C1N=C(S)S2)O)O
2,126,4263,CCOP(=S)(OCC)SCS(=O)(=O)CC
3,129,4215,C1=C(C=C(C(=C1)O)O)CC(=O)[O-]
4,156,4215,C1=CC=C(C=C1)C2=C(C(=CC=C2)O)O


In [45]:
productFilters.merge(affectedByProductFilters, on='simple').groupby('simple').agg({'reaction':'count'})\
    .sort_values(by='reaction', ascending=False)

,reaction
simple,
4282,1495
4215,247
4171,150
4263,73
4164,64
4165,64


In [46]:
POST_REACTION_FILTER = f'{BIN}/postReactionFilter'
PRODUCT_FILTER_MATCHES = f'{TEMP}/productFilterMatches.tsv'

command = f'''{POST_REACTION_FILTER}
    -c {PRODUCT_FILTER_SUBSTRATES}
    -p {PRODUCT_FILTERS}
    -m {PRODUCT_FILTER_MATCHES}'''
ran = subprocess.run(command.split(), capture_output=True)
print(ran.stdout.decode())

#### Excluded Reactions

In [47]:
excluded_rule_reactions = read_csv(REACTANT_FILTER_MATCHES,  sep="\t", header=None, dtype='str')\
                  .append(read_csv(PRODUCT_FILTER_MATCHES, sep="\t", header=None, dtype='str'))
print(excluded_rule_reactions.shape)
excluded_rule_reactions = excluded_rule_reactions.drop_duplicates()
print(excluded_rule_reactions.shape)
excluded_rule_reactions.columns = ['reaction', 'simple']
excluded_rule_reactions = excluded_rule_reactions.merge(
    read_sql('select id, simple from rule', rradb), on='simple'
)
print(excluded_rule_reactions.shape)
excluded_rule_reactions.head()

(1790, 2)
(1749, 2)
(1749, 3)


,reaction,simple,id
0,6289,2982,276
1,6217,2982,276
2,6301,2982,276
3,6718,2982,276
4,1564,2982,276


#### Mark Reactions as Excluded in the Database

In [48]:
excluded_rule_reactions.groupby('id').agg({'reaction':','.join}).apply(
    lambda row:
        rradb.execute(f'''
        update rule_reaction
        set excluded = TRUE
        where reactor = {AMBIT} and rule = {row.name} and reaction in ({row.reaction})
        '''),
    axis=1
);

In [49]:
read_sql('select excluded, count(*) from rule_reaction group by excluded', rradb)

,excluded,count
0,None,8556
1,True,1749
